In [ ]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.exc import IntegrityError
import numpy as np

In [ ]:
engine = create_engine('postgresql:///piwheels')
conn = engine.connect()
meta = MetaData()
meta.bind = conn
meta.reflect(views=True)

In [ ]:
start_date = '2021-04-01'
start_time = '2021-04-01 00:00:00'
end_date = '2021-06-30'
end_time = '2021-07-01 00:00:00'
one_year_ago = '2020-07-01'

In [ ]:
pd.read_sql(f"""
    SELECT COUNT(*) AS downloads_in_period
    FROM downloads
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
""", conn)

In [ ]:
pd.read_sql(f"""
    SELECT COUNT(*) AS downloads_to_date
    FROM downloads
    WHERE accessed_at::date < '{end_date}'
""", conn)

In [ ]:
pd.read_sql(f"""
    SELECT COUNT(*) AS project_page_hits
    FROM project_page_hits
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
""", conn)

In [ ]:
pd.read_sql(f"""
    SELECT COUNT(*) AS project_json_downloads
    FROM project_json_downloads
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
""", conn)

In [ ]:
pd.read_sql(f"""
    SELECT COUNT(*) AS web_page_hits
    FROM web_page_hits
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
""", conn)

In [ ]:
downloads_by_month = pd.read_sql(f"""
    SELECT date_trunc('month', accessed_at) m, COUNT(*) AS c
    FROM downloads
    WHERE accessed_at::date BETWEEN '{one_year_ago}' AND '{end_date}'
    GROUP BY m
    ORDER BY m
""", conn)

downloads_by_month

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)

months = [m.month_name()[:3] for m in downloads_by_month['m']]
downloads = [c for c in downloads_by_month['c']]

colors = ['blue']*9 + ['red']*3
plt.bar(range(12), downloads, color=colors)
plt.title('Downloads by month')
plt.xticks(range(12), months)

plt.savefig('downloads-by-month.png')

plt.show()

In [ ]:
downloads_by_day = pd.read_sql(f"""
    SELECT accessed_at::date AS d, COUNT(*) AS downloads
    FROM downloads
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY d
    ORDER BY d
""", conn)

downloads_by_day

In [ ]:
p = downloads_by_day.plot.bar(title='Downloads by day')
p.figure.savefig('downloads-by-day.png')

In [ ]:
df = pd.read_sql(f"""
    SELECT COUNT(*) AS searches_in_period
    FROM searches
    WHERE accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
""", conn)

searches = df.values[0][0]
searches

In [ ]:
pd.read_sql(f"""
    SELECT date_trunc('month', accessed_at) AS month, COUNT(*) AS searches
    FROM searches
    WHERE accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    GROUP BY month
    ORDER BY month
""", conn)

In [ ]:
searches_by_day = pd.read_sql(f"""
    SELECT accessed_at::date AS day, COUNT(*) AS searches
    FROM searches
    WHERE accessed_at::date BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY day
    ORDER BY day
""", conn)

searches_by_day

In [ ]:
p = searches_by_day.plot.bar(title='Searches by day')
p.figure.savefig('searches-by-day.png')

In [ ]:
pd.read_sql(f"""
    SELECT JUSTIFY_INTERVAL(SUM(
        CASE f.platform_tag
            WHEN 'linux_armv7l' THEN 1                           
            WHEN 'linux_armv6l' THEN 6                                                       
            ELSE 0           
        END *
        CASE                        
            WHEN b.duration > INTERVAL '1 day' THEN INTERVAL '0'
            WHEN b.duration > INTERVAL '6.7 seconds' THEN b.duration - INTERVAL '6.7 seconds'
            ELSE INTERVAL '0'               
        END                  
        )) AS total_time_saved_in_quarter                        
    FROM downloads d
    JOIN files f ON d.filename = f.filename
    JOIN builds b ON b.build_id = f.build_id
    WHERE f.abi_tag <> 'none'
    AND d.accessed_at::date between '{start_date}' and '{end_date}'
""", conn)

In [ ]:
time_saved_by_month = pd.read_sql(f"""
    SELECT date_trunc('month', d.accessed_at) as month, JUSTIFY_INTERVAL(SUM(
        CASE f.platform_tag
            WHEN 'linux_armv7l' THEN 1
            WHEN 'linux_armv6l' THEN 6
            ELSE 0
        END *
        CASE
            WHEN b.duration > INTERVAL '1 day' THEN INTERVAL '0'
            WHEN b.duration > INTERVAL '6.7 seconds' THEN b.duration - INTERVAL '6.7 seconds'
            ELSE INTERVAL '0'
        END
            )) AS time_saved_in_month
    FROM downloads d
    JOIN files f ON d.filename = f.filename
    JOIN builds b ON b.build_id = f.build_id
    WHERE f.abi_tag <> 'none'
    AND d.accessed_at::date between '{one_year_ago}' and '{end_date}'
    GROUP BY month
    ORDER BY month
""", conn)

time_saved_by_month

In [ ]:
months = [m.month_name()[:3] for m in time_saved_by_month['month']]
days_saved = [ts.days for ts in time_saved_by_month['time_saved_in_month']]

colors = ['blue']*9 + ['red']*3
plt.bar(range(12), days_saved, color=colors)
plt.title('Time saved by month')
plt.xticks(range(12), months)

plt.savefig('time-saved-by-month.png')

plt.show()

In [ ]:
top_10_downloads = pd.read_sql(f"""
    select b.package, count(*) c
    from downloads d
    join files f on f.filename = d.filename
    join builds b on b.build_id = f.build_id
    where accessed_at::date between '{start_date}' and '{end_date}'
    group by b.package
    order by c desc
    limit 10;
""", conn)

top_10_downloads

In [ ]:
print("<ol>")
for i, row in top_10_downloads.iterrows():
    print(f"<li><strong><a href='https://www.piwheels.org/project/{row['package']}'>{row['package']}</a></strong> ({row['c']:,})</li>")
print("</ol>")

In [ ]:
top_10_searches = pd.read_sql(f"""
    select package, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    group by package
    order by c desc
    limit 10;
""", conn)

top_10_searches

In [ ]:
print("<ol>")
for pkg, c in top_10_searches.values:
    print(f"<li><strong><a href='https://www.piwheels.org/project/{pkg}'>{pkg}</a></strong> ({c:,})</li>")
print("</ol>")

In [ ]:
top_10_project_pages = pd.read_sql(f"""
    select package, count(*) c
    from project_page_hits
    where accessed_at::date between '{start_date}' and '{end_date}'
    group by package
    order by c desc
    limit 10;
""", conn)

top_10_project_pages

In [ ]:
print("<ol>")
for pkg, c in top_10_project_pages.values:
    print(f"<li><strong><a href='https://www.piwheels.org/project/{pkg}'>{pkg}</a></strong> ({c:,})</li>")
print("</ol>")

In [ ]:
top_10_project_json = pd.read_sql(f"""
    select package, count(*) c
    from project_json_downloads
    where accessed_at::date between '{start_date}' and '{end_date}'
    group by package
    order by c desc
    limit 10;
""", conn)

top_10_project_json

In [ ]:
print("<ol>")
for pkg, c in top_10_project_json.values:
    print(f"<li><strong><a href='https://www.piwheels.org/project/{pkg}'>{pkg}</a></strong> ({c:,})</li>")
print("</ol>")

In [ ]:
pd.read_sql(f"""
    select count(distinct(package)) c
    from project_json_downloads
    where accessed_at::date between '{start_date}' and '{end_date}'
""", conn)

In [ ]:
pd.read_sql(f"""
    select user_agent, count(*) c
    from project_json_downloads
    where accessed_at::date between '{start_date}' and '{end_date}'
    group by user_agent
    order by c desc
""", conn)

In [ ]:
os_usage = pd.read_sql(f"""
    select os_name, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    group by os_name
    order by c desc
""", conn)

os_usage

In [ ]:
linux_usage_pc = 100 * (os_usage.values[0][1] / searches)
print(f'{linux_usage_pc:2.2f}% linux')

In [ ]:
distro_usage = pd.read_sql(f"""
    select distro_name, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    group by distro_name
    order by c desc
    limit 5
""", conn)

distro_usage

In [ ]:
debian_usage = pd.read_sql(f"""
    select count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    and distro_name in ('Raspbian GNU/Linux', 'Debian GNU/Linux')
    order by c desc
    limit 5
""", conn)

debian_usage

In [ ]:
debian_usage_pc = 100 * (debian_usage.c[0] / searches)
print(f'{debian_usage_pc:2.2f}% debian')

In [ ]:
ubuntu_usage = pd.read_sql(f"""
    select count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    and distro_name = 'Ubuntu'
    order by c desc
    limit 5
""", conn)

ubuntu_usage_pc = 100 * (ubuntu_usage.c[0] / searches)
print(f'{ubuntu_usage_pc:2.2f}% ubuntu')

In [ ]:
debian = debian_usage.values[0][0]
others = searches - debian

distro_usage = {
    'Debian / Raspbian / PiOS': debian,
    'Others': others,
}

distro_usage

In [ ]:
plt.pie(distro_usage.values(), labels=distro_usage.keys())
plt.title('Linux distro usage')

plt.savefig('distro-usage.png')

plt.show()

In [ ]:
distro_versions = pd.read_sql(f"""
    select distro_version, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    and distro_name in ('Raspbian GNU/Linux', 'Debian GNU/Linux')
    and distro_version in ('8', '9', '10')
    group by distro_version
    order by c desc
""", conn)

distro_versions

In [ ]:
total = sum(distro_versions.c)

for v in distro_versions.values:
    print(v[0], f'{(100 * v[1] / total):.1f}%')

In [ ]:
version_names = {'10': 'Buster', '9': 'Stretch', '8': 'Jessie'}
plt.pie(distro_versions.c, labels=[version_names[v] for v in distro_versions.distro_version])
plt.title('Raspbian/Debian version usage')

plt.savefig('debian-usage.png')

plt.show()

In [ ]:
arch = pd.read_sql(f"""
    select arch, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    group by arch
    order by c desc
""", conn)

arch

In [ ]:
plt.pie(arch.c, labels=arch.arch)
plt.title('Architecture usage')

plt.savefig('arch.png')

plt.show()

In [ ]:
arm_arch = pd.read_sql(f"""
    select arch, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    and distro_name in ('Raspbian GNU/Linux', 'Debian GNU/Linux')
    and arch in ('armv6l', 'armv7l', 'aarch64')
    group by arch
    order by c desc
""", conn)

total = sum(arm_arch.c)

for v in arm_arch.values:
    print(v[0], f'{(100 * v[1] / total):.1f}%') 

In [ ]:
plt.pie(arm_arch.c, labels=arm_arch.arch)
plt.title('Raspbian/Debian Arm architecture usage')

plt.savefig('debian-arch.png')

plt.show()

In [ ]:
py_vers = pd.read_sql(f"""
    select substring(py_version from 1 for 3) pyver, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and os_name = 'Linux'
    and distro_name in ('Raspbian GNU/Linux', 'Debian GNU/Linux')
    and arch in ('armv6l', 'armv7l', 'aarch64')
    and py_name = 'CPython'
    group by pyver
    order by c desc
    limit 6
""", conn)

py_vers

In [ ]:
plt.pie(py_vers.c, labels=py_vers.pyver)
plt.title('Python version usage')

plt.savefig('py-vers.png')

plt.show()

In [ ]:
total = sum(row[1] for row in py_vers.values)

for ver, c in py_vers.values:
    print(ver, f'{100 * (c / total):.2f}%')

In [ ]:
pip_versions = pd.read_sql(f"""
    select installer_version, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and installer_name = 'pip'
    and installer_version is not null
    group by installer_version
    order by c desc
""", conn)

pip_versions[:10]

In [ ]:
total = sum(v[1] for v in pip_versions.values)
labels = [v[0] if v[1] / total > 0.01 else '' for v in pip_versions.values]

plt.pie(pip_versions.c, labels=labels)
plt.title('pip version usage')

plt.savefig('pip-vers.png')

plt.show()

In [ ]:
setuptools_versions = pd.read_sql(f"""
    select setuptools_version, count(*) c
    from searches
    where accessed_at >= '{start_time}' AND accessed_at < '{end_time}'
    and installer_name = 'pip'
    and setuptools_version is not null
    group by setuptools_version
    order by c desc
""", conn)

setuptools_versions[:10]

In [ ]:
total = sum(v[1] for v in setuptools_versions.values)
labels = [v[0] if v[1] / total > 0.03 else '' for v in setuptools_versions.values]

plt.pie(setuptools_versions.c, labels=labels)
plt.title('setuptools version usage')

plt.savefig('setuptools-vers.png')

plt.show()

In [ ]:
import csv
from glob import glob
from collections import defaultdict

In [ ]:
ONE_TB = 1e12
months = ('202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106')

bandwidth = {}
bandwidth_totals = defaultdict(int)
for month in months:
    bandwidth[month] = defaultdict(int)
    files = glob('/home/ben/piwheels-logs/' + month + '/*-sizes.csv')
    for file in files:
        with open(file) as f:
            r = csv.reader(f)
            for log_type, count in r:
                bandwidth[month][log_type] += int(count)
                bandwidth_totals[month] += int(count) / ONE_TB

In [ ]:
for month, data in bandwidth_totals.items():
    print(month, f'{data:.2f} TB')

In [ ]:
m1, m2, m3 = months[-3:]

bandwidth_in_period = {
    'pip downloads': bandwidth[m1]['LOGDOWNLOAD'] + bandwidth[m2]['LOGDOWNLOAD'] + bandwidth[m3]['LOGDOWNLOAD'],
    'Misc': bandwidth[m1]['OTHER'] + bandwidth[m2]['OTHER'] + bandwidth[m3]['OTHER'],
    'Searches': bandwidth[m1]['LOGSEARCH'] + bandwidth[m2]['LOGSEARCH'] + bandwidth[m3]['LOGSEARCH'],
    '': (
        bandwidth[m1]['LOGPROJECT'] + bandwidth[m2]['LOGPROJECT'] + bandwidth[m3]['LOGPROJECT'] +
        bandwidth[m1]['LOGJSON'] + bandwidth[m2]['LOGJSON'] + bandwidth[m3]['LOGJSON'] +
        bandwidth[m1]['LOGPAGE'] + bandwidth[m2]['LOGPAGE'] + bandwidth[m3]['LOGPAGE']
    ),
}

In [ ]:
plt.pie(bandwidth_in_period.values(), labels=bandwidth_in_period.keys())
plt.title('Bandwidth breakdown')

plt.savefig('bandwidth.png')

plt.show()

In [ ]:
for k, v in bandwidth_in_period.items():
    pc = 100 * (v / sum(bandwidth_in_period.values()))
    print(k, f'{pc:.2f}%')

In [ ]:
bandwidth_by_month = {
    'Jul': bandwidth_totals['202007'],
    'Aug': bandwidth_totals['202008'],
    'Sep': bandwidth_totals['202009'],
    'Oct': bandwidth_totals['202010'],
    'Nov': bandwidth_totals['202011'],
    'Dec': bandwidth_totals['202012'],
    'Jan': bandwidth_totals['202101'],
    'Feb': bandwidth_totals['202102'],
    'Mar': bandwidth_totals['202103'],
    'Apr': bandwidth_totals['202104'],
    'May': bandwidth_totals['202105'],
    'Jun': bandwidth_totals['202106'],
}

colors = ['blue']*9 + ['red']*3
plt.bar(range(len(bandwidth_by_month)), bandwidth_by_month.values(), color=colors)
plt.title('Bandwidth by month (TB)')
plt.xticks(range(len(bandwidth_by_month)), list(bandwidth_by_month.keys()))

plt.savefig('bandwidth-by-month.png')

plt.show()